In [11]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO
import google.generativeai as genai

from dotenv import load_dotenv

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF


# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)


load_dotenv()

#set-up Neo4j and Graphiti 

neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

#Set-up Gemini 

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.5-flash")

class ChatMemory:
    def __init__(self, uri, user, password):
        self.driver = AsyncGraphDatabase.driver(uri, auth=(user, password))
        self.graphiti = Graphiti(uri=uri, user=user, password=password)

    async def save_message_as_episode(self, user_id: str, text: str, sender: str):
        timestamp = datetime.now(timezone.utc).isoformat()

        await self.graphiti.add_episode(
            external_user_id=user_id,
            content=text,
            sender=sender,
            type=EpisodeType.chat,
            timestamp=timestamp,
        )


async def run_conversation():
    memory = ChatMemory(neo4j_uri, neo4j_user, neo4j_password)

    user_id = "user-123"
    user_input = "I need help with my order"

    await memory.save_message_as_episode(user_id, user_input, sender="user")

    response = gemini_model.generate_content(user_input)
    reply = response.text.strip()

    await memory.save_message_as_episode(user_id, reply, sender="agent")

    print("Gemini Reply:", reply)




if __name__ == "__main__":
    asyncio.run(run_conversation())


RuntimeError: asyncio.run() cannot be called from a running event loop